<a href="https://colab.research.google.com/github/daviomce/Gerador-de-Descricoes-de-Produtos-com-IA/blob/main/Gerador_de_Descricoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -q google-generativeai pillow python-dotenv fpdf2
from fpdf import FPDF
import re
import textwrap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import google.generativeai as genai
from google.colab import userdata # Para acessar os segredos
import os

try:
    # Carrega a chave da API do gerenciador de segredos do Colab
    api_key = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = api_key # Opcional, mas algumas libs podem procurar vars de ambiente

    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-1.5-flash') # Ou 'gemini-pro-vision' para versões anteriores da API
    print("API do Gemini configurada com sucesso!")
except Exception as e:
    print(f"Erro ao configurar a API do Gemini. Certifique-se de que a chave 'GOOGLE_API_KEY' está definida nos segredos do Colab.")
    print(f"Detalhes do erro: {e}")
    # Interromper a execução se a API não puder ser configurada
    raise SystemExit("Configuração da API falhou.")

from google.colab import files
from PIL import Image
import io # Para lidar com bytes da imagem
from IPython.display import Image as IPImage, display, Markdown # Importando display e Markdown aqui

In [ ]:
print("Por favor, faça o upload da imagem do produto:")
uploaded_files = files.upload()

pil_image_obj = None
image_bytes = None
if uploaded_files:
    # Pega o nome do primeiro arquivo carregado
    file_name = next(iter(uploaded_files))
    print(f"\nArquivo '{file_name}' carregado com sucesso!")
    image_bytes = uploaded_files[file_name]
    pil_image_obj = Image.open(io.BytesIO(image_bytes))

    # Exibir a imagem (opcional)
    display(IPImage(data=image_bytes, width=300))
else:
    print("Nenhuma imagem foi carregada.")

In [ ]:
print("\n--- Informações Adicionais do Produto ---")
product_name = input("Nome do produto (opcional, deixe em branco se não tiver): ")
brand = input("Marca (opcional): ")
target_audience = input("Público-alvo (opcional): ")
key_features = input("Características chave (não visuais, ex: material, dimensões, funcionalidades internas - opcional): ")

tones = ["Amigável", "Formal", "Técnico", "Divertido", "Premium", "Persuasivo"]
print("\nEscolha o tom desejado para a descrição:")
for i, t in enumerate(tones):
    print(f"{i+1}. {t}")
tone_choice = int(input(f"Digite o número do tom (1-{len(tones)}): ")) -1
desired_tone = tones[tone_choice] if 0 <= tone_choice < len(tones) else tones[0]

print(f"\nTom selecionado: {desired_tone}")

In [5]:
def construct_prompt(image_exists, product_name, brand, target_audience, desired_tone, key_features):
    prompt_parts = []

    if image_exists:
        prompt_parts.append("Analise a imagem fornecida e use-a como base principal para a descrição.")
    else:
        prompt_parts.append("Crie uma descrição de produto.") # Fallback se não houver imagem

    prompt_parts.append(f"O tom da descrição deve ser: {desired_tone}.")

    if product_name:
        prompt_parts.append(f"Nome do produto: {product_name}.")
    if brand:
        prompt_parts.append(f"Marca do produto: {brand}.")
    if target_audience:
        prompt_parts.append(f"O público-alvo principal é: {target_audience}.")
    if key_features:
        prompt_parts.append(f"Considere as seguintes características chave não visíveis ou complementares: {key_features}.")

    prompt_parts.append("\nCrie uma descrição de produto detalhada e otimizada para SEO. A descrição deve ser persuasiva e informativa. Inclua:")
    prompt_parts.append("- Um título atraente e rico em palavras-chave (se o nome do produto não for fornecido, crie um baseado na imagem e características).")
    prompt_parts.append("- Um parágrafo introdutório que capture o interesse e apresente o produto.")
    prompt_parts.append("- Destaque para as principais características e benefícios, tanto os visuais (da imagem) quanto os textuais (das informações fornecidas). Use bullet points para clareza.")
    prompt_parts.append("- Sugestões de possíveis usos ou aplicações do produto.")
    prompt_parts.append("- Argumentos que ressaltem o valor e incentivem a compra, focando nos benefícios para o público-alvo.")
    prompt_parts.append("- Utilização natural de palavras-chave relevantes para o tipo de produto, suas características e seu público-alvo ao longo do texto.")
    prompt_parts.append("- Se houver texto claramente legível na imagem (ex: nome de modelo, slogans curtos), incorpore-o de forma relevante na descrição, se apropriado.")
    prompt_parts.append("\nFormato de saída sugerido (sinta-se à vontade para adaptar para melhor fluidez e SEO):\n**[Título Otimizado para SEO]**\n\n[Parágrafo Introdutório Envolvente]\n\n**Por que você vai amar este produto:**\n* **[Característica/Benefício Chave 1]:** [Breve explicação]\n* **[Característica/Benefício Chave 2]:** [Breve explicação]\n* **[Característica/Benefício Chave 3]:** [Breve explicação]\n\n**Ideal para:**\n[Descreva os usos ou quem se beneficiaria mais]\n\n[Parágrafo final com reforço do valor ou uma chamada para ação sutil].")
    prompt_parts.append("\nPor favor, gere apenas a descrição do produto conforme solicitado.")
    return "\n".join(prompt_parts)

In [6]:
def generate_description_with_gemini(pil_image, text_prompt):
    global model # Garante que estamos usando a variável global 'model'
    if not model:
        print("Erro: O modelo Gemini não foi inicializado.")
        return None
    try:
        generation_config = genai.types.GenerationConfig(
            # candidate_count=1, # Você pode pedir mais de uma se quiser variações
            # max_output_tokens=8192, # Ajuste conforme necessário
            temperature=0.7, # Ajuste para criatividade vs factualidade (0.0 a 1.0+)
            # top_p=1.0,
            # top_k=40
        )

        safety_settings = [ # Ajuste os níveis de segurança conforme sua necessidade
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

        request_contents = [text_prompt]
        if pil_image:
            request_contents.append(pil_image) # Adiciona a imagem à lista de conteúdos

        response = model.generate_content(
            request_contents,
            generation_config=generation_config,
            safety_settings=safety_settings,
            stream=False # Mude para True se quiser processar em streaming
        )

        # Tratamento de blocos e conteúdo
        if response.prompt_feedback and response.prompt_feedback.block_reason:
            print(f"Erro: A geração foi bloqueada. Razão: {response.prompt_feedback.block_reason_message or response.prompt_feedback.block_reason}")
            return None

        if not response.candidates or not response.candidates[0].content or not response.candidates[0].content.parts:
            print("Erro: A API não retornou o conteúdo esperado.")
            # Imprimir a resposta completa para depuração
            # print(f"Resposta completa da API: {response}")
            return None

        return response.text # Acesso direto ao texto combinado

    except Exception as e:
        print(f"Ocorreu um erro ao chamar a API Gemini: {e}")
        # Adicionar log mais detalhado no console para depuração
        # print(f"Erro detalhado da API Gemini: {e}")
        return None

In [19]:
import os
from fpdf import FPDF
import re
import textwrap

def salvar_pdf_no_drive(imagem_bytes, descricao, titulo, caminho_base='/content/drive/MyDrive/Gerador_de_descricao/'):
    """Salva a imagem e a descrição em um arquivo PDF no Google Drive,
    usando o título como nome do arquivo, dentro da pasta 'Gerador_de_descricao'.
    Cria a pasta se ela não existir."""

    # Sanitize o título para criar um nome de arquivo válido
    nome_arquivo = re.sub(r'[^\w\s-]', '', titulo).strip()
    nome_arquivo = re.sub(r'\s+', '_', nome_arquivo) + ".pdf"
    caminho_arquivo = os.path.join(caminho_base, nome_arquivo)

    # Cria o diretório se ele não existir
    os.makedirs(caminho_base, exist_ok=True)  # Cria o diretório e seus pais, se necessário

    pdf = FPDF()
    pdf.add_page()

    # Adiciona a imagem ao PDF
    try:
        from PIL import Image
        import io
        img = Image.open(io.BytesIO(imagem_bytes))
        largura_original, altura_original = img.size
        largura_max = 150
        proporcao = largura_max / largura_original
        altura_ajustada = altura_original * proporcao

        x_imagem = (pdf.w - largura_max) / 2
        pdf.image(io.BytesIO(imagem_bytes), x=x_imagem, y=20, w=largura_max, h=altura_ajustada)
        y_texto = 20 + altura_ajustada + 10
    except Exception as e:
        pdf.cell(0, 10, f"Erro ao adicionar imagem: {e}", ln=True)
        y_texto = 30

    # Adiciona o título, quebrando-o em várias linhas se for muito longo
    pdf.set_font("Arial", 'B', size=16)
    largura_celula = pdf.w - 20  # Margens de 10 em cada lado
    altura_linha = 10
    if pdf.get_string_width(titulo) > largura_celula:
        # Quebra o título em linhas
        linhas = textwrap.wrap(titulo, width=int(largura_celula / pdf.font_size))
        for linha in linhas:
            pdf.cell(0, altura_linha, linha, ln=True, align='C')
    else:
        pdf.cell(0, altura_linha, titulo, ln=True, align='C')
    pdf.ln(5)  # Adiciona um espaço após o título

    # Adiciona a descrição ao PDF
    pdf.set_font("Arial", size=12)
    pdf.set_y(y_texto)
    pdf.multi_cell(0, 10, descricao)

    # Salva o arquivo PDF diretamente no Google Drive
    try:
        pdf.output(caminho_arquivo, "F")
        print(f"✅ Resultado salvo com sucesso em: {caminho_arquivo}")
    except Exception as e:
        print(f"❌ Ocorreu um erro ao salvar o PDF no Drive: {e}")

In [ ]:
if pil_image_obj: # Prosseguir apenas se uma imagem foi carregada
    print("\nConstruindo o prompt...")
    # Se estiver usando os formulários do Colab, use as variáveis _form aqui:
    # product_name_to_use = product_name_form
    # brand_to_use = brand_form
    # ... etc.
    # Senão, use as variáveis do input()
    product_name_to_use = product_name
    brand_to_use = brand
    target_audience_to_use = target_audience
    key_features_to_use = key_features
    desired_tone_to_use = desired_tone

    full_prompt = construct_prompt(
        image_exists=True,
        product_name=product_name_to_use,
        brand=brand_to_use,
        target_audience=target_audience_to_use,
        desired_tone=desired_tone_to_use,
        key_features=key_features_to_use
    )

    print("-----------------------------------------")
    print("Prompt enviado para a IA (para depuração):")
    print(full_prompt)
    print("-----------------------------------------\n")

    print("Gerando descrição... Por favor, aguarde.")
    generated_description = generate_description_with_gemini(pil_image_obj, full_prompt)

    if generated_description:
        print("\n✅ Descrição Gerada:")
        display(Markdown(generated_description))

        # Tenta extrair o título da primeira linha da descrição
        titulo_para_arquivo = generated_description.split('\n')[0].replace('**', '').strip()

        # Chama a função para salvar o PDF no Google Drive
        salvar_pdf_no_drive(image_bytes, generated_description, titulo_para_arquivo)
    else:
        print("\n❌ Não foi possível gerar a descrição.")
elif uploaded_files and not pil_image_obj: # Checagem se o upload deu certo mas a imagem não abriu
    print("\nErro ao processar a imagem carregada. Verifique o formato do arquivo.")
else:
    print("\nNenhuma imagem foi carregada. Por favor, execute a célula de upload de imagem primeiro.")